In [26]:
import pandas as pd

In [27]:
df_pitch_subs = pd.read_csv('pitching_subs.csv')

In [28]:
df_pitch_subs['made_save'] = None
df_pitch_subs['IP'] = 0
df_next_pitcher = df_pitch_subs.shift(-1).reset_index()

# our csv already tells us which pitcher gets the save, so we will leave that calculation to the scorer
# need to check if any relief pitchers either blow,or neither get a S/BS
# for now, let's just see if they blow it based on the state of the game when they come in, and get pulled/end of game
# we know that if they come in with a lead, and leave with the game tied/losing, they blew it
for (cur_index, cur_row), (_, next_row) in zip(df_pitch_subs.iterrows(), df_next_pitcher.iterrows()):
    # made_save will be True(completed save), False(blew save), or None(neither completed or blew)\
    made_save = None
    
    # We will check the state of the game when the pitcher came in (cur_row)
    # as well as the state of the game with the pitcher stopped pitching (next_row)
    
    # make sure next event is the same game, and the same team
    if cur_row['GAME_ID'] == next_row['GAME_ID'] and cur_row['HOME_TEAM'] == next_row['HOME_TEAM']:
        
        # IP will be number if innings pitched
        IP = float(next_row['INN_CT'] - cur_row['INN_CT']) + float((1.0/3.0)*next_row['OUTS_CT']) - float((1.0/3.0)*cur_row['OUTS_CT'])
        
        # home
        if cur_row['HOME_TEAM'] == 1:
            # blew lead if winning in initial state, and losing in their last state
            blew_lead = next_row['HOME_SCORE_CT'] <= next_row['AWAY_SCORE_CT']
            if blew_lead:
                # Definition of Save Opportunity
                # He is credited with at least ⅓ of an inning pitched; and
                #He satisfies one of the following conditions:
                    #1 He enters the game with a lead of no more than three runs and pitches for at least one inning
                    #2 He enters the game, regardless of the count, with the potential tying run either on base, at bat or on deck
                    #3 He pitches for at least three innings.
                    
                if IP > 0:
                    init_score_diff = cur_row['HOME_SCORE_CT'] - cur_row['AWAY_SCORE_CT']
                
                    # save #1 or #2 or #3
                    if (init_score_diff <= 3 and inn_diff >= 1) or (IP >= 3) or (cur_row['HOME_SCORE_CT'] <= cur_row['AWAY_SCORE_CT'] + 2 + sum([cur_row['RUNNER_FIRST'], cur_row['RUNNER_SECOND'], cur_row['RUNNER_THIRD']])):
                        made_save = False    
        #away
        elif cur_row['HOME_TEAM'] == 0:
            # blew lead if winning in initial state, and losing in their last state
            blew_lead = next_row['HOME_SCORE_CT'] >= next_row['AWAY_SCORE_CT']
            if blew_lead:
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['AWAY_SCORE_CT'] - cur_row['HOME_SCORE_CT']
                
                # Save #1 or #2 or #3
                if (init_score_diff <= 3 and IP >= 1) or (IP >= 3) or (cur_row['AWAY_SCORE_CT'] <= cur_row['HOME_SCORE_CT'] + 2 + sum([cur_row['RUNNER_FIRST'], cur_row['RUNNER_SECOND'], cur_row['RUNNER_THIRD']])):
                    made_save = False
    else:
        # calculate IP if last pitcher of game
        IP = cur_row['TOTAL_INN'] - cur_row['INN_CT'] + 1 - float((1.0/3.0)*cur_row['OUTS_CT'])
    # Use Retrosheet to determine if this pitcher was the one who got a save
    if cur_row['PIT_ID'] == cur_row['SAVE_PIT_ID']:
        made_save = True
    
    
    
    # place made_save, IP into the dataframe at the correct index
    df_pitch_subs.iloc[cur_index, df_pitch_subs.columns.get_loc('made_save')] = made_save
    df_pitch_subs.iloc[cur_index, df_pitch_subs.columns.get_loc('IP')] = float(round(IP, 2))

df_pitch_subs.head()

,GAME_ID,EVENT_ID,INN_CT,TOTAL_INN,HOME_TEAM,OUTS_CT,HOME_SCORE_CT,AWAY_SCORE_CT,RUNNER_FIRST,RUNNER_SECOND,RUNNER_THIRD,PIT_ID,SAVE_PIT_ID,FULL_NAME,made_save,IP
0,ANA201004050,54,7,9,1,0,4,3,0,0,0,jepsk001,fuenb001,Kevin Jepsen,None,1.00
1,ANA201004050,64,8,9,1,0,4,3,0,0,0,rodnf001,fuenb001,Fernando Rodney,None,1.00
2,ANA201004050,74,9,9,1,0,6,3,0,0,0,fuenb001,fuenb001,Brian Fuentes,True,1.00
3,ANA201004050,44,5,9,0,2,4,3,1,0,1,craij001,fuenb001,Jesse Crain,False,1.67
4,ANA201004050,61,7,9,0,1,4,3,0,0,0,mijaj001,fuenb001,Jose Mijares,False,0.67


In [29]:
# We already know if a pitcher lost a lead when, but we dont know if they were even entering into a save opportunity
# A Save Opportunity (SvO) happens when a relief pitcher enters the game with a lead
# Above, we didnt check to make sure they were in the lead when they entered the game
def save_sit(row):
    if row['HOME_TEAM'] == 1 and row['HOME_SCORE_CT'] > row['AWAY_SCORE_CT']:
        if row['IP'] > 0:
            init_score_diff = cur_row['HOME_SCORE_CT'] - cur_row['AWAY_SCORE_CT']
            # save #1 or #2 or #3
            if (init_score_diff <= 3 and row['IP'] >= 1) or (row['IP'] >= 3) or (cur_row['HOME_SCORE_CT'] >= cur_row['AWAY_SCORE_CT'] + 2 + sum([cur_row['RUNNER_FIRST'], cur_row['RUNNER_SECOND'], cur_row['RUNNER_THIRD']])):
                return True 
        
    elif row['HOME_TEAM'] == 0 and row['HOME_SCORE_CT'] < row['AWAY_SCORE_CT']:
        if row['IP'] > 0:
            init_score_diff = cur_row['HOME_SCORE_CT'] - cur_row['AWAY_SCORE_CT']
            # save #1 or #2 or #3
            if (init_score_diff <= 3 and row['IP'] >= 1) or (row['IP'] >= 3) or (cur_row['AWAY_SCORE_CT'] >= cur_row['HOME_SCORE_CT'] + 2 + sum([cur_row['RUNNER_FIRST'], cur_row['RUNNER_SECOND'], cur_row['RUNNER_THIRD']])):
                return True 
    return False
df_pitch_subs['SAVE_SIT'] = df_pitch_subs.apply(save_sit, axis=1)

# we know if a pitcher entered into a SvO, so let's disregard all the pitchers who didn't
# we know that all these pitchers are in SvOs and who got the save, we can drop some data that we don't need
df_pitch_subs = df_pitch_subs[df_pitch_subs['SAVE_SIT'] == True].drop(['SAVE_PIT_ID', 'SAVE_SIT'], axis=1).reset_index(drop=True)

df_pitch_subs.head()

,GAME_ID,EVENT_ID,INN_CT,TOTAL_INN,HOME_TEAM,OUTS_CT,HOME_SCORE_CT,AWAY_SCORE_CT,RUNNER_FIRST,RUNNER_SECOND,RUNNER_THIRD,PIT_ID,FULL_NAME,made_save,IP
0,ANA201004050,54,7,9,1,0,4,3,0,0,0,jepsk001,Kevin Jepsen,None,1.0
1,ANA201004050,64,8,9,1,0,4,3,0,0,0,rodnf001,Fernando Rodney,None,1.0
2,ANA201004050,74,9,9,1,0,6,3,0,0,0,fuenb001,Brian Fuentes,True,1.0
3,ANA201004060,68,8,9,0,0,3,5,0,0,0,guerm001,Matt Guerrier,None,1.0
4,ANA201004060,74,9,9,0,0,3,5,0,0,0,raucj001,Jon Rauch,True,1.0


In [30]:
def sit_id(row):
    # '<start_inn(2)><start_out><score_diff><runners_on><IP>'
    score_diff = row['HOME_SCORE_CT'] - row['AWAY_SCORE_CT'] if row['HOME_TEAM'] else row['AWAY_SCORE_CT'] - row['HOME_SCORE_CT']
    runners_on = sum([row['RUNNER_FIRST'], row['RUNNER_SECOND'], row['RUNNER_THIRD']])
    inn = '0' + str(row['INN_CT']) if len(str(row['INN_CT'])) == 1 else str(row['INN_CT'])
    return str(inn) + str(row['OUTS_CT']) + str(score_diff) + str(runners_on) + str(row['IP'])

df_pitch_subs['sit_id'] = df_pitch_subs.apply(sit_id, axis=1)
df_pitch_subs.head()

,GAME_ID,EVENT_ID,INN_CT,TOTAL_INN,HOME_TEAM,OUTS_CT,HOME_SCORE_CT,AWAY_SCORE_CT,RUNNER_FIRST,RUNNER_SECOND,RUNNER_THIRD,PIT_ID,FULL_NAME,made_save,IP,sit_id
0,ANA201004050,54,7,9,1,0,4,3,0,0,0,jepsk001,Kevin Jepsen,None,1.0,070101.0
1,ANA201004050,64,8,9,1,0,4,3,0,0,0,rodnf001,Fernando Rodney,None,1.0,080101.0
2,ANA201004050,74,9,9,1,0,6,3,0,0,0,fuenb001,Brian Fuentes,True,1.0,090301.0
3,ANA201004060,68,8,9,0,0,3,5,0,0,0,guerm001,Matt Guerrier,None,1.0,080201.0
4,ANA201004060,74,9,9,0,0,3,5,0,0,0,raucj001,Jon Rauch,True,1.0,090201.0


In [31]:
df_pitch_subs = df_pitch_subs.drop(df_pitch_subs[(df_pitch_subs.made_save != True) & (df_pitch_subs.made_save != False)].index)
df_pitch_subs

,GAME_ID,EVENT_ID,INN_CT,TOTAL_INN,HOME_TEAM,OUTS_CT,HOME_SCORE_CT,AWAY_SCORE_CT,RUNNER_FIRST,RUNNER_SECOND,RUNNER_THIRD,PIT_ID,FULL_NAME,made_save,IP,sit_id
2,ANA201004050,74,9,9,1,0,6,3,0,0,0,fuenb001,Brian Fuentes,True,1.00,090301.0
4,ANA201004060,74,9,9,0,0,3,5,0,0,0,raucj001,Jon Rauch,True,1.00,090201.0
6,ANA201004070,70,9,9,0,0,1,4,0,0,0,raucj001,Jon Rauch,True,1.00,090301.0
12,ANA201004100,62,8,9,1,0,3,2,0,0,0,shies002,Scot Shields,False,1.00,080101.0
13,ANA201004110,49,7,9,1,0,3,2,0,1,1,jepsk001,Kevin Jepsen,False,1.00,070121.0
14,ANA201004110,53,7,9,0,0,3,4,0,0,0,rosst001,Tyson Ross,True,3.00,070103.0
15,ANA201004190,63,9,9,1,0,2,0,0,0,0,rodnf001,Fernando Rodney,True,1.00,090201.0
17,ANA201004200,79,9,9,1,0,6,5,0,0,0,rodnf001,Fernando Rodney,True,1.00,090101.0
19,ANA201004210,73,9,9,0,0,3,4,0,0,0,valvj001,Jose Valverde,True,1.00,090101.0
22,ANA201004220,84,9,9,0,0,4,5,0,0,0,valvj001,Jose Valverde,True,1.00,090101.0


In [32]:
df_pitch_subs[df_pitch_subs['made_save'] == True].groupby('sit_id').size().to_csv('made.csv')
df_pitch_subs[df_pitch_subs['made_save'] == False].groupby('sit_id').size().to_csv('blown.csv')

In [33]:
df_pitch_subs.groupby('PIT_ID').size()

PIT_ID
aardd001     30
abadf001      2
accaj001      2
aceva001     29
acosm001      4
adamm001      4
adcon001      1
affej001      2
albem001      1
albua001      2
allec002      2
ambrh001      3
andeb004      3
arrej001      1
atchs001      1
aumop001      2
avill001      1
axfoj001     56
ayall001      7
badeb001      1
baila001     63
balfg001     65
bardd001     10
barns002      1
basta001      9
batim001      2
beatp001      1
beavb001      1
bedae001      2
belim001      9
           ... 
tuckr001      1
uehak001     38
valdr002      1
valvj001    121
varva001      1
vasqe001      1
ventj001      2
veraj001     19
villc001      3
vizca001      1
wadec001      1
wagnb001     19
waldj001     36
warra001      1
webbr001      5
weinr001      1
wheed001      3
wilht001     52
willj003      1
wilsb001     38
wilsj004      2
woodb004      3
woodk002     10
workb001      1
wrigj001      4
wrigw001      4
wuerm001      6
zeidj001      1
ziegb001      5
zumaj001      3
Length: 365, dtyp